In [1]:
from astropy.table import Table
import numpy as np
import math
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import matplotlib as mpl
from myfunction import *
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
%matplotlib inline 
%who

FlatLambdaCDM	 L_Halpha_3	 L_Halpha_3_lum	 L_Halpha_4	 L_Halpha_4_lum	 SFR_chabrier	 SkyCoord	 Table	 absmag	 
acos	 acosh	 asin	 asinh	 atan	 atan2	 atanh	 ceil	 constants	 
copysign	 cos	 cosh	 cosmo	 cosmo_cal	 degrees	 degtoHMS	 e	 erf	 
erfc	 errMagFlux	 exp	 expm1	 fabs	 factorial	 floor	 fmod	 frexp	 
fsum	 galcoord	 gamma	 gasmass	 gasmass_no_z	 gcd	 hypot	 inf	 integrate	 
isclose	 isfinite	 isinf	 isnan	 ldexp	 lgamma	 log	 log10	 log1p	 
log2	 math	 modf	 mpl	 myExpFunc	 myLinFunc	 nan	 np	 pi	 
plt	 pow	 radians	 sfe	 sin	 sinh	 sint_err	 sns	 sqrt	 
ssfr	 stellarmass	 tan	 tanh	 tau	 tdep	 trunc	 u	 veltoz	 
veltoz_noerr	 


In [2]:
def vel_hel_lg(v_hel, glon, glat):
    PI = 57.2957795
    Lapex = 93.
    Bapex = -4.
    Vapex = 316.

    dL = glon - Lapex
    T1 = np.sin(glat/PI) * np.sin(Bapex/PI)
    T2 = np.cos(glat/PI) * np.cos(Bapex/PI) * np.cos(dL/PI)

    Vout = v_hel + (Vapex * (T1 + T2) )
    return(Vout)

def vel_CMB(v_hel, glon, glat):
    PI = 57.2957795
    Lapex = 264.14
    Bapex = 48.26
    Vapex = 371.0

    dL = glon - Lapex
    T1 = np.sin(glat/PI) * np.sin(Bapex/PI)
    T2 = np.cos(glat/PI) * np.cos(Bapex/PI) * np.cos(dL/PI)

    Vcmb = v_hel + (Vapex * (T1 + T2) )
    return(Vcmb)

def vel_virgo(Vlg, glon, glat):
    PI = 57.2957795
    Lvir = 284.
    Bvir = 74.
    Vvir = 208. 
    
    dL = Lvir - glon
    T1 = np.sin(Bvir/PI) * np.sin(glat/PI)
    T2 = np.cos(Bvir/PI) * np.cos(glat/PI) * np.cos(dL/PI)
    
    Vcorr = Vlg + (Vvir* (T1 + T2) )
    return(Vcorr)

In [3]:
Dir = 'Users/vaishaliparkash/Dropbox/Office_mac/python_code/Project4_b/'

filename = '2mrs_WISE_dist.csv'
data = Table.read(filename, format = 'csv')
print(data.info())



<Table masked=True length=43533>
    name      dtype  n_bad
------------ ------- -----
        TMID   str16     0
     RA_2MRS float64     0
    DEC_2MRS float64     0
   GLON_2MRS float64     0
   GLAT_2MRS float64     0
         MKC float64     0
         MHC float64     0
         MJC float64     0
        MKTC float64     0
        MHTC float64     0
        MJTC float64     0
         EKC float64     0
         EHC float64     0
         EJC float64     0
        EKTC float64     0
        EHTC float64     0
        EJTC float64     0
         EBV float64     0
        RISO float64     0
        RTOT float64     0
          BA float64     0
       FLAGS    str4     0
        TYPE    str5     0
     TYPESRC    str2     0
           V   int64     0
          EV   int64     0
        VCAT    str1     0
        VSRC   str19     0
       CATID   str28     0
     cntr_01   int64     1
      dist_x float64   118
      pang_x float64   118
     tmid_01   str16     1
       ra_01 float64  

In [4]:
dist_final = np.ma.masked_all(len( data['w2gmag']))
dist_flag = np.ma.masked_all(len( data['w2gmag']))

for ind, gal_dis in enumerate(data['DIST']):
    if np.ma.is_masked(gal_dis) == False:
        dist_final[ind] = gal_dis
        dist_flag[ind] = 1 #Tully
    elif np.ma.is_masked(data['logd_km'][ind]) == False:
        redshift = veltoz_noerr(data['cz_CMB'][ind])
        distance_z = cosmo.luminosity_distance(redshift)
        distance_TF = distance_z / 10**data['logd_km'][ind]
        dist_final[ind] = distance_TF.value
        dist_flag[ind] = 2 #2MTF
    else:
        v_CMB = vel_CMB(data['V'][ind], data['GLON_2MRS'][ind], data['GLAT_2MRS'][ind])
        redshift = veltoz_noerr(v_CMB)
        distance_z = cosmo.luminosity_distance(redshift) 
        dist_final[ind] = distance_z.value
        dist_flag[ind] = 3 #2MRS+CMB       
        
print('done')
        

done


In [12]:
index = 4000
print(data['TMID'][index])
print(dist_final[index], dist_flag[index])
if dist_flag[index] == 1:
    print(data['DIST'][index])
elif dist_flag[index] == 3:
    print(data['V'][index])
    v_CMB = vel_CMB(data['V'][index], data['GLON_2MRS'][index], 
                  data['GLAT_2MRS'][index]) 
    print(v_CMB)
    redshift = veltoz_noerr(v_CMB)
    print(redshift)



09374111-2202064
73.67948544729401 3.0
4718
5047.74434308847
0.016981630187763885


In [13]:
col1 = Table.Column(data=dist_final, name='dist_final')
col2 = Table.Column(data=dist_flag, name='dist_flg')

data.add_column(col1)
data.add_column(col2)

data.write(filename, format = 'csv', overwrite=True)